**Glove Embeddings== 300 Features Extracted**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import logging
import pandas as pd
import numpy as np
import gensim
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re
from bs4 import BeautifulSoup
import csv
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
import os
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
#STOPWORDS =nltk.corpus.stopwords.words('english')

def clean_text(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split())                                                            #if word not in STOPWORDS) # delete stopwors from text
    return text

In [ ]:
#REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
#BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
#STOPWORDS =nltk.corpus.stopwords.words('english')

In [ ]:
path = "/content/drive/MyDrive/newpro/threeclass_problem/glove.6B.300d.txt" 
wv = pd.read_table(path, sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)

In [ ]:
def vec(w):
  return wv.loc[w].as_matrix()

In [ ]:
def w2v_tokenize_text(text):
  tokens = []
  for sent in nltk.sent_tokenize(text, language='english'):
    for word in nltk.word_tokenize(sent, language='english'):
      if len(word) < 2:
        continue
      tokens.append(word)
  return tokens

In [ ]:
def wordprint(words,wv):
    all_words, mean = set(), []
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.index:
           a=np.array(wv.loc[word])
           mean.append(a)
    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(300,)   
    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

In [ ]:
def  word_averaging_listn(wv, text_list):
  return np.vstack([wordprint(post,wv) for post in text_list ])

In [3]:
path = "/content/drive/MyDrive/newpro/threeclass_problem/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [4]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [5]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [6]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [7]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [8]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [9]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [ ]:
df.iloc[:,0] = df.iloc[:,0].apply(clean_text)
print(df.iloc[:100,0]) 

0     there are constants alpha and beta such that f...
1     the length of the segment between the points 2...
2     chewbacca has 20 pieces of cherry gum and 30 p...
3     for some constants a and b let f x left begin ...
4         calculate the sum 1 + 3 + 5 + cdots + 15 + 17
                            ...                        
95    what is the largest value of x that satisfies ...
96    what is the sum of left dfrac 1 3 right + left...
97    on a map a 12centimeter length represents 72 k...
98    find the sum of all values of x such that 2 x2...
99    given that a 0 if f g a 8 where f x x2 + 8 and...
Name: Problem , Length: 100, dtype: object


In [ ]:
comtdata=df

In [ ]:
test_tokenized = comtdata.apply(lambda r: w2v_tokenize_text(r.iloc[0]), axis=1).values

In [ ]:
df_new=pd.DataFrame(test_tokenized)

In [ ]:
df_new.head()

,0
0,"[there, are, constants, alpha, and, beta, such..."
1,"[the, length, of, the, segment, between, the, ..."
2,"[chewbacca, has, 20, pieces, of, cherry, gum, ..."
3,"[for, some, constants, and, let, left, begin, ..."
4,"[calculate, the, sum, cdots, 15, 17]"


In [ ]:
X_comtdata_averagen=word_averaging_listn(wv, test_tokenized)

In [ ]:
X_comtdata_averagen.shape 

(1744, 300)

In [ ]:
x=X_comtdata_averagen 

In [ ]:
X = pd.DataFrame(x)
X

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.018407,0.085011,0.029703,-0.044558,-0.046228,0.002105,-0.017075,0.011753,0.039239,-0.341933,...,0.007392,-0.050867,-0.032619,-0.040177,0.003517,0.009812,0.033037,-0.059005,0.020749,0.040800
1,-0.061130,0.069028,0.029945,-0.075593,-0.055460,0.029755,-0.034010,0.027264,0.033029,-0.427923,...,0.003908,-0.056133,0.006894,0.011421,0.056875,0.074536,0.001674,-0.046282,-0.029717,-0.002265
2,-0.084698,0.059528,-0.019130,-0.054112,-0.039338,0.013001,-0.020171,0.000319,0.021646,-0.397745,...,-0.014100,-0.059806,0.025660,0.011339,0.019782,0.032892,0.005613,-0.029948,-0.029217,-0.031719
3,-0.049269,0.032437,-0.001154,-0.040035,-0.033356,0.029849,0.004792,0.000715,0.013032,-0.474486,...,-0.018463,-0.021215,0.019852,0.006229,0.031585,0.018694,0.036211,-0.089839,-0.007266,-0.009470
4,-0.090736,0.040478,0.054596,-0.015434,-0.090787,0.020753,0.070594,-0.048692,-0.019835,-0.276395,...,0.041861,-0.051594,0.042672,-0.065068,0.078107,0.028808,-0.011964,-0.095961,-0.040278,-0.061897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,-0.032802,0.032271,0.027718,-0.040344,-0.048079,0.005831,-0.022456,0.003891,-0.003708,-0.423163,...,0.016266,-0.019338,0.008811,0.050435,0.038599,0.054819,0.023436,-0.037569,-0.005932,0.002271
1740,-0.035832,0.011162,0.021095,0.011592,-0.076143,0.061837,0.012090,-0.034667,0.019889,-0.157855,...,0.021873,-0.108254,-0.025975,-0.104304,0.031799,0.134799,-0.038208,-0.153331,-0.028206,-0.007938
1741,-0.029652,0.028950,0.040318,-0.066526,-0.016876,0.021540,-0.039522,-0.009164,0.017666,-0.445898,...,0.046150,-0.022066,0.043133,0.026915,0.030850,0.001807,0.035988,-0.016723,0.000472,0.019363
1742,-0.052749,0.067784,0.053815,-0.052505,-0.002247,0.051834,-0.061768,-0.011263,-0.038460,-0.400709,...,-0.014507,0.027017,-0.022949,0.017701,0.037057,0.019114,0.026526,-0.070004,-0.044434,0.002670


In [ ]:
from google.colab import files
X.to_csv('glove_levelsrecode.csv') 
files.download('glove_levelsrecode.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
path = "/content/drive/MyDrive/newpro/threeclass_problem/glove_levelsrecode.csv"
X = pd.read_csv(path)
X.head() 

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,0,-0.018407,0.085011,0.029703,-0.044558,-0.046228,0.002105,-0.017075,0.011753,0.039239,...,0.007392,-0.050867,-0.032619,-0.040177,0.003517,0.009812,0.033037,-0.059005,0.020749,0.040800
1,1,-0.061130,0.069028,0.029945,-0.075593,-0.055460,0.029755,-0.034010,0.027264,0.033029,...,0.003908,-0.056133,0.006894,0.011421,0.056875,0.074536,0.001674,-0.046282,-0.029717,-0.002265
2,2,-0.084698,0.059528,-0.019130,-0.054112,-0.039338,0.013001,-0.020171,0.000319,0.021646,...,-0.014100,-0.059806,0.025660,0.011339,0.019782,0.032892,0.005613,-0.029948,-0.029217,-0.031719
3,3,-0.049269,0.032437,-0.001154,-0.040035,-0.033356,0.029849,0.004792,0.000715,0.013032,...,-0.018463,-0.021215,0.019852,0.006229,0.031585,0.018694,0.036211,-0.089839,-0.007266,-0.009470
4,4,-0.090736,0.040478,0.054596,-0.015434,-0.090787,0.020753,0.070594,-0.048692,-0.019835,...,0.041861,-0.051594,0.042672,-0.065068,0.078107,0.028808,-0.011964,-0.095961,-0.040278,-0.061897


In [11]:
y=df['Level ']

In [12]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 301), (524, 301))

In [13]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test)

Perceptron F1 score: 0.41282934336890365


**Decision Tree**

In [14]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [15]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [16]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [17]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [32]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [33]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

<ipython-input-32-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.631148,0.631148,0.640395,0.62994,0.396947,0.396947,0.405883,0.394907


**Random Forest**

In [34]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [35]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [36]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [37]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    1.0000    1.0000       361
           2     1.0000    1.0000    1.0000       553
           3     1.0000    1.0000    1.0000       306

    accuracy                         1.0000      1220
   macro avg     1.0000    1.0000    1.0000      1220
weighted avg     1.0000    1.0000    1.0000      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.6702    0.4013    0.5020       157
           2     0.5321    0.8397    0.6514       237
           3     0.6964    0.3000    0.4194       130

    accuracy                         0.5744       524
   macro avg     0.6329    0.5136    0.5242       524
weighted avg     0.6142    0.5744    0.5491       524



In [38]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

<ipython-input-32-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.631148,0.631148,0.640395,0.62994,0.396947,0.396947,0.405883,0.394907
1,RandomForest,1.000000,1.000000,1.000000,1.00000,0.574427,0.574427,0.614243,0.549062


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.6301    0.5568    0.5912       361
           2     0.5515    0.6582    0.6002       553
           3     0.6515    0.5131    0.5740       306

    accuracy                         0.5918      1220
   macro avg     0.6110    0.5760    0.5885      1220
weighted avg     0.5998    0.5918    0.5910      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.5906    0.4777    0.5282       157
           2     0.5145    0.6751    0.5839       237
           3     0.5349    0.3538    0.4259       130

    accuracy                         0.5363       524
   macro avg     0.5466    0.5022    0.5127       524
weighted avg     0.5423    0.5363    0.5280       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.610656,0.610656,0.612612,0.610002,0.437023,0.437023,0.435063,0.435058
1,RandomForest,0.982787,0.982787,0.982927,0.982816,0.540076,0.540076,0.554239,0.521777
2,Adaboost,0.591803,0.591803,0.599833,0.590953,0.536260,0.536260,0.542330,0.528029


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9719    0.9584    0.9651       361
           2     0.9594    0.9819    0.9705       553
           3     0.9966    0.9706    0.9834       306

    accuracy                         0.9721      1220
   macro avg     0.9760    0.9703    0.9730      1220
weighted avg     0.9724    0.9721    0.9722      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.5778    0.4968    0.5342       157
           2     0.5619    0.7089    0.6269       237
           3     0.6000    0.4154    0.4909       130

    accuracy                         0.5725       524
   macro avg     0.5799    0.5404    0.5507       524
weighted avg     0.5761    0.5725    0.5654       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.610656,0.610656,0.612612,0.610002,0.437023,0.437023,0.435063,0.435058
1,RandomForest,0.982787,0.982787,0.982927,0.982816,0.540076,0.540076,0.554239,0.521777
2,Adaboost,0.591803,0.591803,0.599833,0.590953,0.536260,0.536260,0.542330,0.528029
3,GBM,0.972131,0.972131,0.972427,0.972163,0.572519,0.572519,0.576097,0.565386


**SVM Model**

In [39]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [40]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [41]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [42]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5181    0.6343    0.5704       361
           2     0.6553    0.2785    0.3909       553
           3     0.4199    0.7451    0.5371       306

    accuracy                         0.5008      1220
   macro avg     0.5311    0.5526    0.4994      1220
weighted avg     0.5557    0.5008    0.4807      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.4847    0.6051    0.5382       157
           2     0.5054    0.1983    0.2848       237
           3     0.3702    0.6692    0.4767       130

    accuracy                         0.4370       524
   macro avg     0.4534    0.4909    0.4333       524
weighted avg     0.4656    0.4370    0.4084       524



In [43]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

<ipython-input-32-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.631148,0.631148,0.640395,0.629940,0.396947,0.396947,0.405883,0.394907
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.574427,0.574427,0.614243,0.549062
2,SVM,0.500820,0.500820,0.555665,0.480657,0.437023,0.437023,0.465647,0.408370


**KNN**

In [ ]:
 knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5775    0.6399    0.6071       361
           2     0.6471    0.6564    0.6517       553
           3     0.6641    0.5621    0.6088       306

    accuracy                         0.6279      1220
   macro avg     0.6296    0.6195    0.6226      1220
weighted avg     0.6307    0.6279    0.6278      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.4599    0.5478    0.5000       157
           2     0.5273    0.4895    0.5077       237
           3     0.5385    0.4846    0.5101       130

    accuracy                         0.5057       524
   macro avg     0.5085    0.5073    0.5059       524
weighted avg     0.5099    0.5057    0.5060       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.610656,0.610656,0.612612,0.610002,0.437023,0.437023,0.435063,0.435058
1,RandomForest,0.982787,0.982787,0.982927,0.982816,0.540076,0.540076,0.554239,0.521777
2,Adaboost,0.591803,0.591803,0.599833,0.590953,0.536260,0.536260,0.542330,0.528029
3,GBM,0.972131,0.972131,0.972427,0.972163,0.572519,0.572519,0.576097,0.565386
4,SVM,0.489344,0.489344,0.588316,0.386894,0.480916,0.480916,0.578196,0.366708
5,KNN,0.627869,0.627869,0.630749,0.627756,0.505725,0.505725,0.509860,0.505975


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4841    0.3795    0.4255       361
           2     0.5474    0.1881    0.2799       553
           3     0.3280    0.8007    0.4653       306

    accuracy                         0.3984      1220
   macro avg     0.4531    0.4561    0.3902      1220
weighted avg     0.4736    0.3984    0.3695      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.5278    0.3631    0.4302       157
           2     0.4545    0.1266    0.1980       237
           3     0.3114    0.8385    0.4542       130

    accuracy                         0.3740       524
   macro avg     0.4313    0.4427    0.3608       524
weighted avg     0.4410    0.3740    0.3311       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.610656,0.610656,0.612612,0.610002,0.437023,0.437023,0.435063,0.435058
1,RandomForest,0.982787,0.982787,0.982927,0.982816,0.540076,0.540076,0.554239,0.521777
2,Adaboost,0.591803,0.591803,0.599833,0.590953,0.536260,0.536260,0.542330,0.528029
3,GBM,0.972131,0.972131,0.972427,0.972163,0.572519,0.572519,0.576097,0.565386
4,SVM,0.489344,0.489344,0.588316,0.386894,0.480916,0.480916,0.578196,0.366708
5,KNN,0.627869,0.627869,0.630749,0.627756,0.505725,0.505725,0.509860,0.505975
6,GNB,0.398361,0.398361,0.473620,0.369505,0.374046,0.374046,0.440981,0.331130
